In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scanpy as sc
import numpy as np
import pandas as pd
from matplotlib.pyplot import rc_context
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 200)
#sc.logging.print_versions() 
sc.logging.print_header()
sc.settings.verbosity = 3  
sc.settings.set_figure_params(dpi=120, color_map='viridis')

scanpy==1.8.2 anndata==0.8.0 umap==0.5.2 numpy==1.21.0 scipy==1.8.0 pandas==1.4.2 scikit-learn==0.24.2 statsmodels==0.13.2 python-igraph==0.9.9 pynndescent==0.5.6


In [2]:
adata = sc.read('/home/jovyan/mounting_point/renal_covid_19_gex_myeloid_soupX_QC_orig.ident_annotated_levle23.h5ad',cache=True) #annotated data

In [ ]:
sc.pl.umap(adata,neighbors_key='X_pca_harmony_orig.ident',color=['annot4'], legend_fontsize = 7,legend_loc='on data')

In [ ]:
list(adata.obs['annot4'].unique())

In [ ]:
'C139' in list(adata.obs['sample_id_broad'].unique())

In [ ]:
'C116' in list(adata.obs['sample_id_broad'].unique())

In [ ]:
'C141' in list(adata.obs['sample_id_broad'].unique())

In [ ]:
#remove C141 from the samples
adata = adata[~adata.obs['sample_id_broad'].isin(['C141'])]

In [ ]:
'C141' in list(adata.obs['sample_id_broad'].unique())

In [ ]:
#get monocyte only
monocyte = adata[adata.obs['annot4'].isin(['CD14mono_activated','CD16mono','CD14mono','Int.mono','CD16mono_C1','CD16mono_IFN','CD14mono_IFN','CD14mono_anti_inflammatory'])]

In [ ]:
list(monocyte.obs['annot4'].unique())

In [ ]:
list(monocyte.obs['case_control'].unique())

In [ ]:
#remove 'nan' from case_control
monocyte = monocyte[monocyte.obs['case_control']!='nan']

In [ ]:
list(monocyte.obs['case_control'].unique())

In [ ]:
#getting p,n,r in wave 1 and wave 2

monocyte.obs['wave_case_control']='nan'
monocyte.obs.loc[(monocyte.obs["centre"]=='NCL') &(monocyte.obs["case_control"]=='POSITIVE'), "wave_case_control"] = 'wave1_positive'
monocyte.obs.loc[(monocyte.obs["centre"]=='NCL') &(monocyte.obs["case_control"]=='NEGATIVE'), "wave_case_control"] = 'wave1_negative'
monocyte.obs.loc[(monocyte.obs["centre"]=='NCL') &(monocyte.obs["case_control"]=='RECOVERY'), "wave_case_control"] = 'wave1_recovery'

monocyte.obs.loc[(monocyte.obs["centre"]=='Cambridge') &(monocyte.obs["case_control"]=='POSITIVE'), "wave_case_control"] = 'wave2_positive'
monocyte.obs.loc[(monocyte.obs["centre"]=='Cambridge') &(monocyte.obs["case_control"]=='NEGATIVE'), "wave_case_control"] = 'wave2_negative'
monocyte.obs.loc[(monocyte.obs["centre"]=='Cambridge') &(monocyte.obs["case_control"]=='RECOVERY'), "wave_case_control"] = 'wave2_recovery'

In [ ]:
list(monocyte.obs['wave_case_control'].unique()) #wave 1 doesn't have recovery samples

In [ ]:
#test = monocyte[monocyte.obs['wave_case_control']=='wave2_recovery']
#print (list(test.obs['case_control'].unique()))
#print (list(test.obs['centre'].unique()))

In [ ]:
df = pd.crosstab(monocyte.obs['wave_case_control'],monocyte.obs['annot4'])
df

In [ ]:
df.columns = df.columns.add_categories(['wave'])
df['wave'] = df.index # making index into column
df

In [ ]:
p1=df.plot(x='wave', kind='bar', stacked=True,
        title='distribution of monocytes in Wave 1 and Wave 2')
p1.set_ylabel("num of cells")
p1.set_xlabel(" ")
plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p1.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p1.text(x+width/2, 
            y+height/2, 
            '{:.0f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7)  

In [ ]:
#getting percentage
cols = list(df.columns)
cols = cols[:-1]
df[cols] = df[cols].div(df[cols].sum(axis=1), axis=0).multiply(100)

In [ ]:
df

In [ ]:
df.sum(axis=1)

In [ ]:
monocyte.obs['wave_case_control'].value_counts()

In [ ]:
p2 = df.plot(x='wave', kind='bar', stacked=True,
        title='distribution of monocytes in Wave1 and Wave2',figsize=(5,5))
p2.set_ylabel('percentage')
p2.set_xlabel('')
plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p2.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p2.text(x+width/2, 
            y+height/2, 
            '{:.1f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7)  




p2.text(-0.2,101,'23749',fontsize=9)
p2.text(0.8,101,'22846',fontsize=9)
p2.text(1.8,101,'5364',fontsize=9)
p2.text(2.8,101,'34215',fontsize=9)
p2.text(3.8,101,'6008',fontsize=9)

# Getting wave 2 only 

In [ ]:
monocyte_wave1 = monocyte[monocyte.obs['centre']=='NCL']
monocyte_wave2 = monocyte[monocyte.obs['centre']=='Cambridge']

In [ ]:
monocyte_wave2.obs['case_control'].unique()

In [ ]:
# adding steroid status and case_control conditions

monocyte_wave2.obs['steroid_status']='nan'

monocyte_wave2.obs.loc[(monocyte_wave2.obs["sample_id_broad"].isin(['C146',
'C139',
'C147',
'C145',
'C138',
'C33',
'C101',
'C141'])) &(monocyte_wave2.obs['case_control']=='POSITIVE'), "steroid_status"] = 'steroid_positive'

monocyte_wave2.obs.loc[(monocyte_wave2.obs["sample_id_broad"].isin(['C146',
'C139',
'C147',
'C145',
'C138',
'C33',
'C101',
'C141'])) &(monocyte_wave2.obs['case_control']=='NEGATIVE'), "steroid_status"] = 'steroid_negative'

monocyte_wave2.obs.loc[(monocyte_wave2.obs["sample_id_broad"].isin(['C146',
'C139',
'C147',
'C145',
'C138',
'C33',
'C101',
'C141'])) &(monocyte_wave2.obs['case_control']=='RECOVERY'), "steroid_status"] = 'steroid_recovery'

#####################

monocyte_wave2.obs.loc[(monocyte_wave2.obs["sample_id_broad"].isin(['C169',
'C170',
'C168',
'C108',
'C58',
'C140',
'C137',
'C187',
'C190']))&(monocyte_wave2.obs['case_control']=='POSITIVE'), "steroid_status"] = 'no_steroid_positive'

monocyte_wave2.obs.loc[(monocyte_wave2.obs["sample_id_broad"].isin(['C169',
'C170',
'C168',
'C108',
'C58',
'C140',
'C137',
'C187',
'C190']))&(monocyte_wave2.obs['case_control']=='NEGATIVE'), "steroid_status"] = 'no_steroid_negative'


monocyte_wave2.obs.loc[(monocyte_wave2.obs["sample_id_broad"].isin(['C169',
'C170',
'C168',
'C108',
'C58',
'C140',
'C137',
'C187',
'C190']))&(monocyte_wave2.obs['case_control']=='RECOVERY'), "steroid_status"] = 'no_steroid_recovery'



In [ ]:
list(monocyte_wave2.obs['steroid_status'].unique())

In [ ]:
#test = monocyte_wave2[monocyte_wave2.obs['steroid_status']=='steroid_negative']
#print (test.obs['case_control'].unique())

In [ ]:
df = pd.crosstab(monocyte_wave2.obs['steroid_status'],monocyte_wave2.obs['annot4'])
df

In [ ]:
df.columns = df.columns.add_categories(['steroid_status'])
df['steroid_status'] = df.index # making index into column
df

In [ ]:
p1=df.plot(x='steroid_status', kind='bar', stacked=True,
        title='distribution of monocytes in different steroid status')
p1.set_xlabel('')
p1.set_ylabel('number of cells')

plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p1.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p1.text(x+width/2, 
            y+height/2, 
            '{:.0f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7)  

In [ ]:
cols = list(df.columns)
cols = cols[:-1]
df[cols] = df[cols].div(df[cols].sum(axis=1), axis=0).multiply(100)

In [ ]:
df.sum(axis=1)

In [ ]:
monocyte_wave2.obs['steroid_status'].value_counts()

In [ ]:
p2 = df.plot(x='steroid_status', kind='bar', stacked=True,
        title='distribution of monocytes in different steroid status',figsize=(5,5))
plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
p2.set_xlabel('')
p2.set_ylabel('percentage')
for p in p2.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p2.text(x+width/2, 
            y+height/2, 
            '{:.1f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7)  


p2.text(-0.2,101,'4107',fontsize=9)
p2.text(0.8,101,'21413',fontsize=9)
p2.text(1.8,101,'5033',fontsize=9)
p2.text(2.8,101,'1257',fontsize=9)
p2.text(3.8,101,'12802',fontsize=9)
p2.text(4.8,101,'975',fontsize=9)

# steroid only

In [ ]:
# to get sub-samples who got steroid 
steroid = monocyte_wave2[monocyte_wave2.obs['sample_id_broad'].isin(['C146',
'C139',
'C147',
'C145',
'C138',
'C33',
'C101',
'C141'])]


no_steroid = monocyte_wave2[monocyte_wave2.obs['sample_id_broad'].isin(['C169',
'C170',
'C168',
'C108',
'C58',
'C140',
'C137',
'C187',
'C190'])]


In [ ]:
(list(steroid.obs['case_control'].unique()))

In [ ]:
# adding metdata about before and after steroid samples based on date of steroid injection
steroid.obs['steroid_timeline']='nan'

steroid.obs.loc[(steroid.obs["sample_id"].isin(['C138_230121','C138_280121','C138_260121','C101_060121',
                                                                     'C141_020221','C141_200121','C141_300121','C141_220121',
                                                                      'C141_260121','C146_120121','C146_160121','C146_140121',
                                                                      'C146_190121','C147_140121','C33_140121','C33_190121','C33_160121']))&(steroid.obs['case_control']=='POSITIVE'), "steroid_timeline"] = 'before_steroid_pos'
steroid.obs.loc[(steroid.obs["sample_id"].isin(['C138_230121','C138_280121','C138_260121','C101_060121',
                                                                     'C141_020221','C141_200121','C141_300121','C141_220121',
                                                                      'C141_260121','C146_120121','C146_160121','C146_140121',
                                                                      'C146_190121','C147_140121','C33_140121','C33_190121','C33_160121']))&(steroid.obs['case_control']=='NEGATIVE'), "steroid_timeline"] = 'before_steroid_neg'
steroid.obs.loc[(steroid.obs["sample_id"].isin(['C138_230121','C138_280121','C138_260121','C101_060121',
                                                                     'C141_020221','C141_200121','C141_300121','C141_220121',
                                                                      'C141_260121','C146_120121','C146_160121','C146_140121',
                                                                      'C146_190121','C147_140121','C33_140121','C33_190121','C33_160121']))&(steroid.obs['case_control']=='RECOVERY'), "steroid_timeline"] = 'before_steroid_rec'

##########################

steroid.obs.loc[(steroid.obs["sample_id"].isin(['C138_040221','C138_020221','C138_300121','C138_300420',
                                                                    'C101_110121','C101_250221','C101_200121','C101_130121',
                                                                    'C101_180121','C141_090321','C145_230121','C145_190121',
                                                                    'C145_070121','C145_140121','C145_120121','C145_300420','C146_250221','C146_270121',
                                                                     'C146_230121','C146_300420','C147_280121','C147_260121','C147_190121','C147_230121','C147_300420',
                                                                     'C33_260121']))&(steroid.obs['case_control']=='POSITIVE'), "steroid_timeline"] = 'after_steroid_pos'
steroid.obs.loc[(steroid.obs["sample_id"].isin(['C138_040221','C138_020221','C138_300121','C138_300420',
                                                                    'C101_110121','C101_250221','C101_200121','C101_130121',
                                                                    'C101_180121','C141_090321','C145_230121','C145_190121',
                                                                    'C145_070121','C145_140121','C145_120121','C145_300420','C146_250221','C146_270121',
                                                                     'C146_230121','C146_300420','C147_280121','C147_260121','C147_190121','C147_230121','C147_300420',
                                                                     'C33_260121']))&(steroid.obs['case_control']=='NEGATIVE'), "steroid_timeline"] = 'after_steroid_neg'

steroid.obs.loc[(steroid.obs["sample_id"].isin(['C138_040221','C138_020221','C138_300121','C138_300420',
                                                                    'C101_110121','C101_250221','C101_200121','C101_130121',
                                                                    'C101_180121','C141_090321','C145_230121','C145_190121',
                                                                    'C145_070121','C145_140121','C145_120121','C145_300420','C146_250221','C146_270121',
                                                                     'C146_230121','C146_300420','C147_280121','C147_260121','C147_190121','C147_230121','C147_300420',
                                                                     'C33_260121']))&(steroid.obs['case_control']=='RECOVERY'), "steroid_timeline"] = 'after_steroid_rec'






In [ ]:
steroid.obs['steroid_timeline'].unique()

In [ ]:
#test = steroid[steroid.obs['steroid_timeline']=='after_steroid_neg']
#print(test.obs['case_control'].unique())

In [ ]:
df = pd.crosstab(steroid.obs['steroid_timeline'],steroid.obs['annot4'])
df

In [ ]:
df.columns = df.columns.add_categories(['steroid_timeline'])
df['steroid_timeline'] = df.index # making index into column

In [ ]:
df

In [ ]:
df=df.reindex(["before_steroid_pos", "after_steroid_pos", "after_steroid_rec",'after_steroid_neg'])
df

In [ ]:
p1=df.plot(x='steroid_timeline', kind='bar', stacked=True,
        title='distribution of monocytes before and after steroid')
p1.set_ylabel('num of cells')
p1.set_xlabel('')

plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p1.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p1.text(x+width/2, 
            y+height/2, 
            '{:.0f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7) 

In [ ]:
cols = list(df.columns)
cols = cols[:-1]
df[cols] = df[cols].div(df[cols].sum(axis=1), axis=0).multiply(100)

In [ ]:
steroid.obs['steroid_timeline'].value_counts()

In [ ]:
p2 = df.plot(x='steroid_timeline', kind='bar', stacked=True,
        title='distribution of monocytes before and after steroid',figsize=(5,5))
p2.set_xlabel('')
p2.set_ylabel('percentage')
plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p2.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p2.text(x+width/2, 
            y+height/2, 
            '{:.1f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7) 



p2.text(-0.2,101,'4879',fontsize=9)
p2.text(0.8,101,'7923',fontsize=9)
p2.text(1.8,101,'975',fontsize=9)
p2.text(2.8,101,'1257',fontsize=9)

# adding metadata

In [ ]:
meta_obs=pd.read_csv('/home/jovyan/data/covid_19_renal/obs_myeloid.csv')

In [ ]:
mdata = sc.read('/home/jovyan/data/covid_19_renal/renal_covid_19_gex_myeloid_soupX_QC.h5ad',cache=True)

In [ ]:
mdata.obs['WHO_temp_severity'] = meta_obs['WHO_temp_severity'].values.astype("str") 
mdata.obs['WHO_temp_severity'] = mdata.obs['WHO_temp_severity'].astype('category')

In [ ]:
steroid.obs['WHO_temp_severity'] = steroid.obs['sex'].astype('object')

In [ ]:
steroid.obs['WHO_temp_severity'].update(pd.Series(mdata.obs['WHO_temp_severity'].astype('object')))

In [ ]:
steroid.obs['WHO_temp_severity'] = steroid.obs['WHO_temp_severity'].astype('category')

In [ ]:
steroid.obs['WHO_temp_severity'].unique()

#  steroid with positive case only -divided into 4 severity 

In [ ]:
steroid.obs['steroid_timeline'].unique()

In [ ]:
steroid_pos_only = steroid[steroid.obs['steroid_timeline'].isin(['before_steroid_pos','after_steroid_pos'])]

In [ ]:
steroid_pos_only.obs['WHO_temp_severity'].unique()

In [ ]:
# adding severity and steroid meta
steroid_pos_only.obs['steroid_severity']='nan'

steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='before_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='mild'), "steroid_severity"] = 'before_steroid_pos_mild'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='before_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='moderate'), "steroid_severity"] = 'before_steroid_pos_moderate'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='before_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='severe'), "steroid_severity"] = 'before_steroid_pos_severe'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='before_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='critical'), "steroid_severity"] = 'before_steroid_pos_critical'

steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='after_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='mild'), "steroid_severity"] = 'after_steroid_pos_mild'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='after_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='moderate'), "steroid_severity"] = 'after_steroid_pos_moderate'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='after_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='severe'), "steroid_severity"] = 'after_steroid_pos_severe'
steroid_pos_only.obs.loc[(steroid_pos_only.obs["steroid_timeline"]=='after_steroid_pos')&(steroid_pos_only.obs['WHO_temp_severity']=='critical'), "steroid_severity"] = 'after_steroid_pos_critical'


In [ ]:
steroid_pos_only.obs['steroid_severity'].unique()

In [ ]:
#test = steroid_pos_only[steroid_pos_only.obs['steroid_severity']=='after_steroid_pos_severe']
#print(test.obs['case_control'].unique())
#print(test.obs['WHO_temp_severity'].unique())

In [ ]:
df = pd.crosstab(steroid_pos_only.obs['steroid_severity'],steroid_pos_only.obs['annot4'])
df

In [ ]:
df.columns = df.columns.add_categories(['steroid_severity'])
df['steroid_severity'] = df.index 
df

In [ ]:
df=df.reindex(["before_steroid_pos_mild", "after_steroid_pos_mild", "before_steroid_pos_moderate",'after_steroid_pos_moderate','before_steroid_pos_severe','after_steroid_pos_severe',
              'after_steroid_pos_critical'])
df

In [ ]:
p1=df.plot(x='steroid_severity', kind='bar', stacked=True,
        title='distribution of monocytes before and after steroid and severity')
p1.set_ylabel('num of cells')
p1.set_xlabel('')

plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p1.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p1.text(x+width/2, 
            y+height/2, 
            '{:.0f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7) 

In [ ]:
cols = list(df.columns)
cols = cols[:-1]
df[cols] = df[cols].div(df[cols].sum(axis=1), axis=0).multiply(100)

In [ ]:
df

In [ ]:
steroid_pos_only.obs['steroid_severity'].value_counts()

In [ ]:
p2 = df.plot(x='steroid_severity', kind='bar', stacked=True,
        title='distribution of monocytes before and after steroid and severity',figsize=(5,5))
p2.set_xlabel('')
p2.set_ylabel('percentage')
plt.grid(False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
for p in p2.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy() 
        p2.text(x+width/2, 
            y+height/2, 
            '{:.1f}'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',rotation=0,size=7) 



p2.text(-0.2,101,'2678',fontsize=9)
p2.text(0.8,101,'1811',fontsize=9)
p2.text(1.8,101,'1911',fontsize=9)
p2.text(2.8,101,'2146',fontsize=9)
p2.text(3.8,101,'290',fontsize=9)
p2.text(4.8,101,'1226',fontsize=9)
p2.text(5.8,101,'2740',fontsize=9)

In [ ]:
save_file = '/home/jovyan/scripts/renal_covid_19/steroid_pipeline/steroid_pos_only.h5ad'
steroid_pos_only.write_h5ad(save_file)